In [1]:
from os import path, makedirs
import glob
import json
from jsonschema import validate, Draft7Validator
from tempfile import TemporaryDirectory
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Alphabet import generic_protein
from sys import exit
import datetime
import time
import re
import urllib3
import certifi
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [2]:
def fetch_gbk(nucl_acc, email, clean_cache = False):
    mibig_gbks_folder = "../../preprocessed/gbks_not_in_ncbi/"
    cache_folder = "../../preprocessed/cached_gbks/"
    cache_path = path.join(cache_folder, "{}.gbk".format(nucl_acc))

    if clean_cache or not path.exists(cache_path) or path.getsize(cache_path) < 100:
        if re.match("^MIBIG:BGC\d{7}\.\d+$", nucl_acc): # fetch from gbk folder
            gbk_path = path.join(mibig_gbks_folder, "{}.gbk".format(nucl_acc.split(":")[1]))
            return open(gbk_path, "r")
        else: # (re)download from ncbi
            Entrez.email = email
            handle = Entrez.efetch(db="nucleotide", id=nucl_acc, rettype="gbwithparts", retmode="text")
            if not path.exists(cache_folder):
                makedirs(cache_folder)
        with open(cache_path, "w") as gbk_file:
            gbk_file.write(handle.read())
            
    return open(cache_path, "r")

# from antismash
def get_aa_translation(seq_record, feature):
    """Obtain content for translation qualifier for specific CDS feature in sequence record"""
    extracted = feature.extract(seq_record.seq).ungap('-')

    # ensure the extracted section is a multiple of three by trimming any excess
    if len(extracted) % 3 != 0:
        extracted = extracted[:-(len(extracted) % 3)]

    fasta_seq = extracted.translate(to_stop=True)
    if len(fasta_seq) == 0:
        print("Retranslating {} with stop codons".format(feature.id))
        fasta_seq = extracted.translate()

    # replace ambiguous aminos with an explicit unknown
    string_version = str(fasta_seq)
    for bad in "*BJOUZ":
        string_version = string_version.replace(bad, "X")

    # and remove any gaps
    string_version = string_version.replace("-", "")
    fasta_seq = Seq(string_version, generic_protein)

    return fasta_seq

def get_aa_sequence(feature, to_stop=False):
    """Extract sequence from specific CDS feature in sequence record"""
    fasta_seq = feature.qualifiers['translation'][0]
    if "*" in fasta_seq:
        if to_stop:
            fasta_seq = fasta_seq.split('*')[0]
        else:
            fasta_seq = fasta_seq.replace("*","X")
    if "-" in fasta_seq:
        fasta_seq = fasta_seq.replace("-","")
    return fasta_seq

In [3]:
def fetch_mibig_final_gbk(bgc_id, clean_cache = False):
    cache_folder = "../../preprocessed/cached_mibig_finalgbks/"
    cache_path = path.join(cache_folder, "{}.1.final.gbk".format(bgc_id))
    
    if clean_cache or not path.exists(cache_path) or path.getsize(cache_path) < 100:
        mibig_url = "https://mibig.secondarymetabolites.org/repository/{}/{}.1.final.gbk".format(bgc_id, bgc_id)
        resp = http.request('GET', mibig_url)
        with open(cache_path, "w") as cf:
            cf.write(resp.data.decode('utf-8', 'ignore'))
        
    return open(cache_path, "r")

In [4]:
with open("../../preprocessed/gene_counts.tsv", "w") as o:
    o.write("mibig_id\tnucl_acc\tloc\tfrom_gbk\tfrom_genefinding\tnew_annot\tre_annot\n")

with open("../../preprocessed/.tsv", "w") as o:
    o.write("")
    
def fetch_additional_genes(annotation_data):
    # fetch seq records from ncbi (or use cached files)
    nucl_acc = annotation_data["general_params"]["loci"]["accession"]
    #if re.match("^MIBIG:BGC\d{7}\.\d+$", nucl_acc.upper()): # using mibig accessions
    #    return
    nucl_acc_to_downloads = [nucl_acc]

    # fetch sequence gbks
    email = annotation_data["personal"]["submitter_email"] # probably not a good idea, but let's just put it as it is for now
    nucl_records = {}
    for nucl_acc_to_download in nucl_acc_to_downloads:
        num_try = 1
        clean_cache = False
        while num_try < 6:
            try:
                with fetch_gbk(nucl_acc_to_download, email, clean_cache) as gbk_handle:
                    seq_record = SeqIO.read(gbk_handle, "genbank") # the gbk should contains only 1 file
                    if len(seq_record.seq) < 1:
                        raise Exception("Empty sequence record {}".format(nucl_acc_to_download))
                    nucl_records[nucl_acc_to_download] = seq_record
                    break
            except:
                print("Error...")
                clean_cache = True
            num_try += 1
            time.sleep(5)
        if nucl_acc_to_download not in nucl_records: # failed to download NCBI data
            print("{} Failed to download: http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={}&rettype=gbwithparts&retmode=text".format(annotation_data["general_params"]["mibig_accession"], nucl_acc_to_download))
            return {}

    # fetch finalgbks
    bgc_id = annotation_data["general_params"]["mibig_accession"]
    mibig_genes = {}
    with fetch_mibig_final_gbk(bgc_id) as mibig_gbk_handle:
        mibig_seq_records = SeqIO.parse(mibig_gbk_handle, "genbank")
        for mibig_seq_record in mibig_seq_records:
            match = re.search("(between(?P<start>\d+)-(?P<end>\d+)ntfrom){0,1}GenBankID(?P<acc>[A-Z0-9_\.]+)\.", mibig_seq_record.annotations['comment'].replace(" ", "").replace("\n", ""))
            if match:
                nucl_accession = match.group("acc").upper()
                if match.group("start"):
                    cl_start = int(match.group("start"))
                    cl_end = int(match.group("end"))
                    if nucl_accession not in mibig_genes:
                        mibig_genes[nucl_accession] = []
                    for feature in mibig_seq_record.features:
                        if feature.type == "CDS":
                            f_start = cl_start - 1 + feature.location.start
                            f_end = cl_start - 1 + feature.location.end
                            feature.location = FeatureLocation(f_start, f_end)
                            mibig_genes[nucl_accession].append(feature)
                else:
                    mibig_genes[nucl_accession] = [feature for feature in mibig_seq_record.features if feature.type == "CDS"]
            else:
                print(mibig_seq_record.annotations['comment'])
        
    # compare with mibig clustergbk, then
    #for loci in annotation_data["general_params"]["loci"]["nucl_acc"]:
    loci = annotation_data["general_params"]["loci"]
    if True:
        gene_counts = 0
        seq_record = nucl_records[loci["accession"].upper()]        
        record = {
            "nucl_acc": loci["accession"],
            "start_pos": loci["start_coord"] if "start_coord" in loci and loci["start_coord"] > 0 else 1, # use genbank-offset
            "end_pos": loci["end_coord"] if "end_coord" in loci and loci["end_coord"] > 0 else len(seq_record.seq) # use genbank-offset
        }
        genes_in_gbk = {}
        for feature in seq_record.features:
            if feature.type == "CDS" and feature.location.start >= record["start_pos"]-1 and feature.location.end <= record["end_pos"]:
                new_gene = {}
                # name
                if "gene" in feature.qualifiers:
                    new_gene["gene_name"] = feature.qualifiers["gene"][0]
                elif "protein_id" in feature.qualifiers:
                    new_gene["gene_name"] = feature.qualifiers["protein_id"][0]
                elif "locus_tag" in feature.qualifiers:
                    new_gene["gene_name"] = feature.qualifiers["locus_tag"][0]
                # id
                if "locus_tag" in feature.qualifiers:
                    new_gene["gene_id"] = feature.qualifiers["locus_tag"][0]
                elif "protein_id" in feature.qualifiers:
                    new_gene["gene_id"] = feature.qualifiers["protein_id"][0]
                elif "gene" in feature.qualifiers:
                    new_gene["gene_id"] = feature.qualifiers["gene"][0]
                # holder for everything (for matching purpose only)
                new_gene["ids"] = []
                if "locus_tag" in feature.qualifiers:
                    new_gene["ids"].append(feature.qualifiers["locus_tag"][0].lower())
                if "protein_id" in feature.qualifiers:
                    new_gene["ids"].append(feature.qualifiers["protein_id"][0].lower())
                if "gene" in feature.qualifiers:
                    new_gene["ids"].append(feature.qualifiers["gene"][0].lower())
                # locs
                new_gene["gene_startpos"] = feature.location.start + 1
                new_gene["gene_endpos"] = feature.location.end
                # strand
                new_gene["strand"] = feature.location.strand
                # annotation
                if "product" in feature.qualifiers:
                    new_gene["gene_annotation"] = feature.qualifiers["product"][0]
                # set aa sequence
                if "translation" in feature.qualifiers:
                    new_gene["aa_seq"] = str(get_aa_sequence(feature, False))
                else: # translate aa sequence from nucleotide sequence
                    new_gene["aa_seq"] = str(get_aa_translation(seq_record, feature))
                # dummy -- todo: set this based on ..?
                new_gene["gene_function"] = "Unknown"
                gene = (feature.location.start, feature.location.end) # use locus as identifier
                genes_in_gbk[gene] = new_gene
        
        genes_from_mibig = {}
        if len(genes_in_gbk.keys()) < 1: # fetch genefinder's genes from mibig
            mibig_features = mibig_genes[loci["accession"].upper()]
            for feature in mibig_features:
                gene = (feature.location.start, feature.location.end) # use locus as identifier
                if feature.type == "CDS" and feature.location.start >= record["start_pos"]-1 and feature.location.end <= record["end_pos"]:
                    if gene not in genes_in_gbk:
                        genes_from_mibig[gene] = feature
                        ## append to annotation json
                        ## todo: check existing annotations
                        if "genes" not in annotation_data["general_params"]:
                            annotation_data["general_params"]["genes"] = {"gene": []}
                        elif "gene" not in annotation_data["general_params"]["genes"]:
                            annotation_data["general_params"]["genes"]["gene"] = []
                        new_gene = {}
                        # name
                        if "gene" in feature.qualifiers:
                            new_gene["gene_name"] = feature.qualifiers["gene"][0]
                        elif "protein_id" in feature.qualifiers:
                            new_gene["gene_name"] = feature.qualifiers["protein_id"][0]
                        elif "locus_tag" in feature.qualifiers:
                            new_gene["gene_name"] = feature.qualifiers["locus_tag"][0]
                        # id
                        if "locus_tag" in feature.qualifiers:
                            new_gene["gene_id"] = feature.qualifiers["locus_tag"][0]
                        elif "protein_id" in feature.qualifiers:
                            new_gene["gene_id"] = feature.qualifiers["protein_id"][0]
                        elif "gene" in feature.qualifiers:
                            new_gene["gene_id"] = feature.qualifiers["gene"][0]
                        # locs
                        new_gene["gene_startpos"] = feature.location.start + 1
                        new_gene["gene_endpos"] = feature.location.end
                        # strand
                        new_gene["strand"] = feature.location.strand
                        # annotation
                        if "product" in feature.qualifiers:
                            new_gene["gene_annotation"] = feature.qualifiers["product"][0]
                        # set aa sequence
                        if "translation" in feature.qualifiers:
                            new_gene["aa_seq"] = str(get_aa_sequence(feature, False))
                        else: # translate aa sequence from nucleotide sequence
                            new_gene["aa_seq"] = str(get_aa_translation(seq_record, feature))
                        # dummy -- todo: set this based on ..?
                        new_gene["gene_function"] = "Unknown"
                        # update if exist, add if not
                        is_updated = False                        
                        for annot_gene in annotation_data["general_params"]["genes"]["gene"]:
                            if "gene_id" in annot_gene and new_gene["gene_id"] == annot_gene["gene_id"]:
                                annot_gene["gene_startpos"] = new_gene["gene_startpos"]
                                annot_gene["gene_endpos"] = new_gene["gene_endpos"]
                                if "gene_name" not in annot_gene and "gene_name" in new_gene:
                                    annot_gene["gene_name"] = new_gene["gene_name"]
                                if "gene_annotation" not in annot_gene and "gene_annotation" in new_gene:
                                    annot_gene["gene_annotation"] = new_gene["gene_annotation"]
                                if "aa_seq" not in annot_gene and "aa_seq" in new_gene:
                                    annot_gene["aa_seq"] = new_gene["aa_seq"]
                        if not is_updated:
                            annotation_data["general_params"]["genes"]["gene"].append(new_gene)
        
        # validate annotated genes
        reannotation_count = 0
        newannotation_count = 0
        newannotated_genes = []
        annot_genes = []
        if "genes" in annotation_data["general_params"] and "gene" in annotation_data["general_params"]["genes"]:
            for annot_gene in annotation_data["general_params"]["genes"]["gene"]:
                corresponding_gbk_gene = None
                for key in genes_in_gbk:
                    gbk_gene = genes_in_gbk[key]
                    if (("gene_id" in annot_gene and annot_gene["gene_id"].lower() in gbk_gene["ids"])
                        or ("gene_name" in annot_gene and annot_gene["gene_name"].lower() in gbk_gene["ids"])):
                        corresponding_gbk_gene = gbk_gene
                        break
                gbk_gene = corresponding_gbk_gene
                if gbk_gene != None:
                    if "gene_id" in gbk_gene:
                        annot_gene["gene_id"] = gbk_gene["gene_id"]
                    if "gene_name" in gbk_gene:
                        annot_gene["gene_name"] = gbk_gene["gene_name"]
                    # check if gene_id or gene_name is in gbk, then discard positional information (?)
                    if "gene_startpos" in annot_gene:
                        del annot_gene["gene_startpos"]
                    if "gene_endpos" in annot_gene:
                        del annot_gene["gene_endpos"]
                    if "strand" in annot_gene:
                        del annot_gene["strand"]
                    # check if there is any point on putting the annotation
                    if "gene_annotation" in annot_gene and ("gene_annotation" in gbk_gene and annot_gene["gene_annotation"].lower() != gbk_gene["gene_annotation"].lower()) or ("gene_annotation" not in gbk_gene):
                        annot_genes.append(annot_gene)
                        reannotation_count += 1
                    elif "gene_function" in annot_gene and ("gene_function" in gbk_gene and annot_gene["gene_function"].lower() != gbk_gene["gene_function"].lower()) or ("gene_function" not in gbk_gene):
                        annot_genes.append(annot_gene)
                        reannotation_count += 1
                    elif "aa_seq" in annot_gene and ("aa_seq" in gbk_gene and annot_gene["aa_seq"] != gbk_gene["aa_seq"]) or ("aa_seq" not in gbk_gene):
                        annot_genes.append(annot_gene)
                        reannotation_count += 1
                    else:
                        for key in annot_gene:
                            if key not in gbk_gene:
                                annot_genes.append(annot_gene)
                                reannotation_count += 1
                                break
                else:
                    # check start&endpos, if not within range of record then discard positional information
                    # if startpos = endpos, also discard positional information
                    if "gene_startpos" in annot_gene:
                        if "gene_endpos" in annot_gene:
                            if ((annot_gene["gene_startpos"] < record["start_pos"]) or (annot_gene["gene_endpos"] > record["end_pos"])
                               or (annot_gene["gene_startpos"] == annot_gene["gene_endpos"])):
                                del annot_gene["gene_startpos"]
                                del annot_gene["gene_endpos"]
                            elif (annot_gene["gene_startpos"] > annot_gene["gene_endpos"]):
                                stpos = annot_gene["gene_startpos"]
                                annot_gene["gene_startpos"] = annot_gene["gene_endpos"]
                                annot_gene["gene_endpos"] = stpos
                        else:
                            del annot_gene["gene_startpos"]
                    elif "gene_endpos" in annot_gene:
                        del annot_gene["gene_endpos"]
                    annot_genes.append(annot_gene)
                    if "gene_id" in annot_gene and re.match("^ctg\\d+_orf\\d+$", annot_gene["gene_id"]):
                        continue
                    newannotation_count += 1
                    gn = ""
                    if "gene_id" in annot_gene:
                        gn = "{}/".format(annot_gene["gene_id"])
                    else:
                        gn = "-/"
                    if "gene_name" in annot_gene:
                        gn = "{}{}".format(gn, annot_gene["gene_name"])
                    else:
                        gn = "{}-".format(gn)
                    newannotated_genes.append(gn)

            annotation_data["general_params"]["genes"]["gene"] = annot_genes            
        
        if newannotation_count > 0:
            with open("../../preprocessed/new_annotated_genes.tsv", "a") as o:
                o.write("{}\t{}\n".format(annotation_data["general_params"]["mibig_accession"], ";".join(newannotated_genes)))
        
        with open("../../preprocessed/gbks_gene_counts.tsv", "a") as o:
            report_row = ("{}\t{}\t{}-{}\t{}\t{}\t{}\t{}".format(annotation_data["general_params"]["mibig_accession"], record["nucl_acc"], record["start_pos"], record["end_pos"], len(genes_in_gbk.keys()), len(genes_from_mibig.keys()), newannotation_count, reannotation_count))
            o.write("{}\n".format(report_row))

In [ ]:
input_path = "../../preprocessed/json_2.0_phase_5/"
output_folder = "../../preprocessed/json_2.0_genefinding/"
for json_path in sorted(glob.glob(path.join(input_path, "BGC*.json"))):
    with open(json_path, "r") as json_file:
        bgc_id = path.basename(json_path).split(".")[0]
        data = json.load(json_file)
        print("Scanning {}".format(bgc_id))
        #fetch_additional_genes(data)
        with open(path.join(output_folder, "{}.json".format(bgc_id)), "w") as o:
            o.write(json.dumps(data, indent=4, separators=(',', ': ')))

print("All data fetched!")

Scanning BGC0000001
Scanning BGC0000002
Scanning BGC0000003
Scanning BGC0000004
Scanning BGC0000005
Scanning BGC0000006
Scanning BGC0000007
Scanning BGC0000008
Scanning BGC0000009
Scanning BGC0000010
Scanning BGC0000011
Scanning BGC0000012
Scanning BGC0000013
Scanning BGC0000014
Scanning BGC0000015
Scanning BGC0000016
Scanning BGC0000017
Scanning BGC0000018
Scanning BGC0000019
Scanning BGC0000020
Scanning BGC0000021
Scanning BGC0000022
Scanning BGC0000023
Scanning BGC0000024
Scanning BGC0000025
Scanning BGC0000026
Scanning BGC0000027
Scanning BGC0000028
Scanning BGC0000029
Scanning BGC0000030
Scanning BGC0000031
Scanning BGC0000032
Scanning BGC0000033
Scanning BGC0000034
Scanning BGC0000035
Scanning BGC0000036
Scanning BGC0000037
Scanning BGC0000038
Error...
Scanning BGC0000039
Scanning BGC0000040
Scanning BGC0000041
Scanning BGC0000042
Scanning BGC0000043
Scanning BGC0000044
Scanning BGC0000045
Scanning BGC0000046
Scanning BGC0000047
Scanning BGC0000048
Scanning BGC0000049
Scanning BG

Scanning BGC0000417
Scanning BGC0000418
Scanning BGC0000419
Scanning BGC0000420
Scanning BGC0000421
Scanning BGC0000422
Scanning BGC0000423
Scanning BGC0000424
Scanning BGC0000425
Scanning BGC0000426
Scanning BGC0000427
Scanning BGC0000428
Scanning BGC0000429
Scanning BGC0000430
Scanning BGC0000431
Scanning BGC0000432
Scanning BGC0000433
Scanning BGC0000434
Scanning BGC0000435
Scanning BGC0000436
Scanning BGC0000437
Scanning BGC0000438
Scanning BGC0000439
Scanning BGC0000440
Scanning BGC0000441
Scanning BGC0000442
Scanning BGC0000443
Scanning BGC0000444
Scanning BGC0000445
Scanning BGC0000446
Scanning BGC0000447
Scanning BGC0000448
Scanning BGC0000449
Scanning BGC0000450
Scanning BGC0000451
Scanning BGC0000452
Scanning BGC0000453
Scanning BGC0000454
Scanning BGC0000455
Scanning BGC0000456
Scanning BGC0000457
Scanning BGC0000458
Scanning BGC0000459
Scanning BGC0000460
Scanning BGC0000461
Scanning BGC0000462
Scanning BGC0000463
Scanning BGC0000464
Scanning BGC0000465
Scanning BGC0000466


In [ ]:
gene_counts = {}
with open("../../preprocessed/gbks_gene_counts.tsv", "r") as o:
    i = 0
    for line in o:
        if i == 0:
            continue
        i += 1
        line = line.rstrip()
        tabs = line.split("\t")
        bgcid = tabs[0]
        if bgcid not in gene_counts:
            gene_counts[bgcid] = [0, 0, 0, 0] # total loci, total genes from acc, total genes from mibig, total genes
        gene_counts[bgcid][0] += 1
        gene_counts[bgcid][1] += int(tabs[3])
        gene_counts[bgcid][2] += int(tabs[4])
        gene_counts[bgcid][3] += int(tabs[3]) + int(tabs[4])
        
with open("../../preprocessed/gbks_gene_counts_merged.tsv", "w") as o:
    o.write("bgc_id\ttotal_loci\tfrom_acc\tfrom_mibig\ttotal\n")
    for bgcid in sorted(gene_counts.keys()):
        ar = gene_counts[bgcid]
        o.write("{}\t{}\t{}\t{}\t{}\n".format(bgcid, ar[0], ar[1], ar[2], ar[3]))